# Data Exploration - GTA Real Estate Hotspots

**Author:** Kyle Williamson (Data Engineer)  
**Date:** 2024-11-06  
**Purpose:** Initial exploration of collected data sources

## Objectives
1. Load and validate all data sources
2. Understand data structure and quality
3. Identify missing values and outliers
4. Generate summary statistics
5. Create initial visualizations

## Data Sources
- Toronto Open Data: Real estate, building permits, demographics
- OpenStreetMap: Road networks, POIs, transit stations

In [ ]:
# Standard libraries
import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Geospatial
import geopandas as gpd
import folium
from folium import plugins

# Network analysis
import networkx as nx

# Add src to path
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.append(str(PROJECT_ROOT / 'src'))

# Set paths
DATA_RAW = PROJECT_ROOT / 'data' / 'raw'
DATA_PROCESSED = PROJECT_ROOT / 'data' / 'processed'
RESULTS = PROJECT_ROOT / 'results'

# Create results directories
(RESULTS / 'figures').mkdir(parents=True, exist_ok=True)
(RESULTS / 'tables').mkdir(parents=True, exist_ok=True)

# Plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Imports successful")
print(f"Project root: {PROJECT_ROOT}")
print(f"Data directory: {DATA_RAW}")

## 1. Real Estate Data Exploration

Let's start by loading and exploring the real estate/housing data from Toronto Open Data.

In [ ]:
# Find the most recent real estate file
real_estate_dir = DATA_RAW / 'real_estate'

if not real_estate_dir.exists():
    print("Real estate data not found. Run data collection first:")
    print("   python src/data_collection.py --sources real_estate")
else:
    # Get most recent file
    csv_files = list(real_estate_dir.glob('*.csv'))
    
    if csv_files:
        latest_file = max(csv_files, key=lambda p: p.stat().st_mtime)
        print(f"✓ Loading: {latest_file.name}")
        
        df_real_estate = pd.read_csv(latest_file)
        
        print(f"\n Dataset Shape: {df_real_estate.shape}")
        print(f"   Rows: {df_real_estate.shape[0]:,}")
        print(f"   Columns: {df_real_estate.shape[1]}")
        
        print(f"\n Columns:")
        for i, col in enumerate(df_real_estate.columns, 1):
            print(f"   {i:2d}. {col}")
        
        print(f"\n First few rows:")
        display(df_real_estate.head())
    else:
        print("No CSV files found in real_estate directory")

In [ ]:
if 'df_real_estate' in locals():
    print("=" * 60)
    print("DATA QUALITY ASSESSMENT")
    print("=" * 60)
    
    # Missing values
    print("\n1. Missing Values:")
    missing = df_real_estate.isnull().sum()
    missing_pct = (missing / len(df_real_estate)) * 100
    missing_df = pd.DataFrame({
        'Missing_Count': missing,
        'Missing_Percentage': missing_pct
    }).sort_values('Missing_Count', ascending=False)
    
    print(missing_df[missing_df['Missing_Count'] > 0])
    
    # Data types
    print("\n2. Data Types:")
    print(df_real_estate.dtypes)
    
    # Summary statistics
    print("\n3. Summary Statistics:")
    display(df_real_estate.describe())
    
    # Check for duplicates
    duplicates = df_real_estate.duplicated().sum()
    print(f"\n4. Duplicate Rows: {duplicates}")
    
    # Date range (if Year column exists)
    if 'Year' in df_real_estate.columns:
        print(f"\n5. Year Range: {df_real_estate['Year'].min()} - {df_real_estate['Year'].max()}")

## 2. Real Estate Visualizations

Let's visualize trends in the real estate data.

In [ ]:
if 'df_real_estate' in locals() and 'Year' in df_real_estate.columns:
    # Check if we have price columns
    price_cols = [col for col in df_real_estate.columns if 'price' in col.lower() or 'value' in col.lower()]
    
    if price_cols:
        print(f"Found price columns: {price_cols}")
        
        # Create time series plot
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Plot 1: Average trends over time
        if len(price_cols) > 0:
            price_col = price_cols[0]
            yearly_avg = df_real_estate.groupby('Year')[price_col].mean()
            
            axes[0].plot(yearly_avg.index, yearly_avg.values, marker='o', linewidth=2, markersize=8)
            axes[0].set_title(f'Average {price_col} Over Time', fontsize=14, fontweight='bold')
            axes[0].set_xlabel('Year', fontsize=12)
            axes[0].set_ylabel(price_col, fontsize=12)
            axes[0].grid(True, alpha=0.3)
            
            # Add trend line
            z = np.polyfit(yearly_avg.index, yearly_avg.values, 1)
            p = np.poly1d(z)
            axes[0].plot(yearly_avg.index, p(yearly_avg.index), "--", alpha=0.5, color='red', label='Trend')
            axes[0].legend()
        
        # Plot 2: Distribution of prices
        if len(price_cols) > 0:
            axes[1].hist(df_real_estate[price_col].dropna(), bins=30, edgecolor='black', alpha=0.7)
            axes[1].set_title(f'Distribution of {price_col}', fontsize=14, fontweight='bold')
            axes[1].set_xlabel(price_col, fontsize=12)
            axes[1].set_ylabel('Frequency', fontsize=12)
            axes[1].grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        plt.savefig(RESULTS / 'figures' / 'real_estate_trends.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"\n✓ Figure saved to: {RESULTS / 'figures' / 'real_estate_trends.png'}")
    else:
        print("No price columns found for visualization")

## 3. Building Permits Data

Let's load and explore development activity through building permits.

In [ ]:
# Find the most recent building permits file
permits_dir = DATA_RAW / 'permits'

if not permits_dir.exists():
    print("Building permits data not found. Run data collection first:")
    print("   python src/data_collection.py --sources permits")
else:
    csv_files = list(permits_dir.glob('*.csv'))
    
    if csv_files:
        latest_file = max(csv_files, key=lambda p: p.stat().st_mtime)
        print(f"✓ Loading: {latest_file.name}")
        
        # Load with low_memory=False to avoid dtype warnings
        df_permits = pd.read_csv(latest_file, low_memory=False)
        
        print(f"\n Dataset Shape: {df_permits.shape}")
        print(f"   Rows: {df_permits.shape[0]:,}")
        print(f"   Columns: {df_permits.shape[1]}")
        
        print(f"\n Key Columns:")
        key_cols = ['ISSUED_DATE', 'PERMIT_TYPE', 'WORK_TYPE', 'LATITUDE', 'LONGITUDE', 
                    'ESTIMATED_VALUE', 'PROPOSED_BUILDING_TYPE']
        for col in key_cols:
            if col in df_permits.columns:
                print(f"   ✓ {col}")
            else:
                print(f"   ✗ {col} (not found)")
        
        print(f"\n First few rows:")
        display(df_permits.head())
        
        # Convert date column if exists
        date_cols = [col for col in df_permits.columns if 'DATE' in col.upper()]
        if date_cols:
            for col in date_cols:
                try:
                    df_permits[col] = pd.to_datetime(df_permits[col], errors='coerce')
                    print(f"✓ Converted {col} to datetime")
                except:
                    print(f"⚠ Could not convert {col} to datetime")
    else:
        print(" No CSV files found in permits directory")

In [ ]:
if 'df_permits' in locals():
    print("=" * 60)
    print("BUILDING PERMITS ANALYSIS")
    print("=" * 60)
    
    # Check for spatial data
    has_coordinates = 'LATITUDE' in df_permits.columns and 'LONGITUDE' in df_permits.columns
    
    if has_coordinates:
        valid_coords = df_permits[['LATITUDE', 'LONGITUDE']].notna().all(axis=1).sum()
        print(f"\n1. Spatial Coverage:")
        print(f"   Records with valid coordinates: {valid_coords:,} ({valid_coords/len(df_permits)*100:.1f}%)")
    
    # Permit types
    if 'PERMIT_TYPE' in df_permits.columns:
        print(f"\n2. Permit Types:")
        permit_counts = df_permits['PERMIT_TYPE'].value_counts()
        print(permit_counts.head(10))
    
    # Temporal distribution
    if date_cols:
        date_col = date_cols[0]
        df_permits['Year'] = df_permits[date_col].dt.year
        print(f"\n3. Temporal Distribution:")
        print(f"   Date range: {df_permits[date_col].min()} to {df_permits[date_col].max()}")
        print(f"\n   Permits by year:")
        print(df_permits['Year'].value_counts().sort_index())
    
    # Estimated values
    if 'ESTIMATED_VALUE' in df_permits.columns:
        print(f"\n4. Construction Values:")
        # Convert to numeric if needed
        df_permits['ESTIMATED_VALUE_NUM'] = pd.to_numeric(
            df_permits['ESTIMATED_VALUE'], errors='coerce'
        )
        
        values_summary = df_permits['ESTIMATED_VALUE_NUM'].describe()
        print(values_summary)
        
        total_value = df_permits['ESTIMATED_VALUE_NUM'].sum()
        print(f"\n   Total estimated construction value: ${total_value:,.0f}")

In [ ]:
if 'df_permits' in locals() and 'Year' in df_permits.columns:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Plot 1: Permits over time
    yearly_permits = df_permits['Year'].value_counts().sort_index()
    axes[0, 0].bar(yearly_permits.index, yearly_permits.values, color='steelblue', alpha=0.7)
    axes[0, 0].set_title('Building Permits Over Time', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Year')
    axes[0, 0].set_ylabel('Number of Permits')
    axes[0, 0].grid(True, alpha=0.3, axis='y')
    
    # Plot 2: Permit types
    if 'PERMIT_TYPE' in df_permits.columns:
        top_types = df_permits['PERMIT_TYPE'].value_counts().head(10)
        axes[0, 1].barh(range(len(top_types)), top_types.values, color='coral', alpha=0.7)
        axes[0, 1].set_yticks(range(len(top_types)))
        axes[0, 1].set_yticklabels(top_types.index, fontsize=9)
        axes[0, 1].set_title('Top 10 Permit Types', fontsize=14, fontweight='bold')
        axes[0, 1].set_xlabel('Count')
        axes[0, 1].grid(True, alpha=0.3, axis='x')
    
    # Plot 3: Construction values over time (if available)
    if 'ESTIMATED_VALUE_NUM' in df_permits.columns:
        yearly_value = df_permits.groupby('Year')['ESTIMATED_VALUE_NUM'].sum() / 1e9  # Convert to billions
        axes[1, 0].plot(yearly_value.index, yearly_value.values, marker='o', linewidth=2, color='green')
        axes[1, 0].set_title('Total Construction Value Over Time', fontsize=14, fontweight='bold')
        axes[1, 0].set_xlabel('Year')
        axes[1, 0].set_ylabel('Value (Billions $)')
        axes[1, 0].grid(True, alpha=0.3)
    
    # Plot 4: Value distribution
    if 'ESTIMATED_VALUE_NUM' in df_permits.columns:
        # Filter out extreme outliers for better visualization
        values = df_permits['ESTIMATED_VALUE_NUM'].dropna()
        q99 = values.quantile(0.99)
        values_filtered = values[values <= q99]
        
        axes[1, 1].hist(values_filtered, bins=50, color='purple', alpha=0.7, edgecolor='black')
        axes[1, 1].set_title('Distribution of Construction Values (99th percentile)', fontsize=14, fontweight='bold')
        axes[1, 1].set_xlabel('Estimated Value ($)')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig(RESULTS / 'figures' / 'building_permits_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n Figure saved to: {RESULTS / 'figures' / 'building_permits_analysis.png'}")

## 4. Spatial Visualization - Building Permits Map

Let's create an interactive map showing the geographic distribution of building permits.

In [ ]:
if 'df_permits' in locals() and has_coordinates:
    print("Creating interactive map of building permits...")
    
    # Filter to valid coordinates within GTA bounds
    # GTA approximate bounds: Lat 43.5-44.0, Lon -79.8 to -79.0
    df_map = df_permits[
        (df_permits['LATITUDE'].notna()) & 
        (df_permits['LONGITUDE'].notna()) &
        (df_permits['LATITUDE'] > 43.5) & 
        (df_permits['LATITUDE'] < 44.0) &
        (df_permits['LONGITUDE'] > -79.8) & 
        (df_permits['LONGITUDE'] < -79.0)
    ].copy()
    
    print(f" Filtered to {len(df_map):,} permits within GTA bounds")
    
    if len(df_map) > 0:
        # For performance, sample if dataset is too large
        if len(df_map) > 5000:
            print(f" Sampling 5,000 permits for map performance")
            df_map = df_map.sample(n=5000, random_state=42)
        
        # Create base map centered on Toronto
        toronto_center = [43.7, -79.4]
        m = folium.Map(
            location=toronto_center,
            zoom_start=11,
            tiles='OpenStreetMap'
        )
        
        # Add marker cluster for better performance
        from folium.plugins import MarkerCluster
        marker_cluster = MarkerCluster(name='Building Permits').add_to(m)
        
        # Add markers
        for idx, row in df_map.iterrows():
            # Create popup text
            popup_text = f"""
            <b>Permit Type:</b> {row.get('PERMIT_TYPE', 'N/A')}<br>
            <b>Date:</b> {row.get('ISSUED_DATE', 'N/A')}<br>
            <b>Value:</b> ${row.get('ESTIMATED_VALUE', 'N/A'):,} <br>
            <b>Building Type:</b> {row.get('PROPOSED_BUILDING_TYPE', 'N/A')}
            """
            
            folium.Marker(
                location=[row['LATITUDE'], row['LONGITUDE']],
                popup=folium.Popup(popup_text, max_width=300),
                icon=folium.Icon(color='blue', icon='info-sign')
            ).add_to(marker_cluster)
        
        # Add layer control
        folium.LayerControl().add_to(m)
        
        # Save map
        map_file = RESULTS / 'figures' / 'permits_map.html'
        m.save(str(map_file))
        
        print(f"\n Interactive map saved to: {map_file}")
        print(f" Open in browser to view: file://{map_file.absolute()}")
        
        # Display in notebook
        display(m)
    else:
        print(" No valid coordinates found for mapping")
else:
    print(" Building permits data or coordinates not available for mapping")

## 5. Data Summary and Export

Let's create summary tables for the midterm report.

In [ ]:
# Create comprehensive summary table
summary_data = []

# Real Estate Summary
if 'df_real_estate' in locals():
    summary_data.append({
        'Data Source': 'Real Estate / Housing',
        'Records': f"{len(df_real_estate):,}",
        'Date Range': f"{df_real_estate['Year'].min()}-{df_real_estate['Year'].max()}" if 'Year' in df_real_estate.columns else 'N/A',
        'Spatial Granularity': 'FSA / Neighbourhood',
        'Status': ' Complete',
        'Notes': f"{df_real_estate.shape[1]} columns"
    })

# Building Permits Summary
if 'df_permits' in locals():
    date_range = 'N/A'
    if 'Year' in df_permits.columns:
        date_range = f"{df_permits['Year'].min()}-{df_permits['Year'].max()}"
    
    summary_data.append({
        'Data Source': 'Building Permits',
        'Records': f"{len(df_permits):,}",
        'Date Range': date_range,
        'Spatial Granularity': 'Lat/Lon coordinates',
        'Status': ' Complete',
        'Notes': f"{valid_coords:,} with valid coordinates"
    })

# Demographics (placeholder)
demographics_dir = DATA_RAW / 'demographics'
if demographics_dir.exists() and list(demographics_dir.glob('*.csv')):
    summary_data.append({
        'Data Source': 'Demographics (Census)',
        'Records': 'TBD',
        'Date Range': '2021',
        'Spatial Granularity': 'DA / Neighbourhood',
        'Status': ' In Progress',
        'Notes': 'Data collected, needs processing'
    })
else:
    summary_data.append({
        'Data Source': 'Demographics (Census)',
        'Records': 'N/A',
        'Date Range': 'N/A',
        'Spatial Granularity': 'DA / Neighbourhood',
        'Status': ' Not Started',
        'Notes': 'Pending collection'
    })

# Transit Networks (placeholder)
transit_dir = DATA_RAW / 'transit'
if transit_dir.exists() and list(transit_dir.glob('*.graphml')):
    summary_data.append({
        'Data Source': 'Road Network (OSM)',
        'Records': 'Network Graph',
        'Date Range': '2024',
        'Spatial Granularity': 'Node/Edge level',
        'Status': ' In Progress',
        'Notes': 'Network downloaded'
    })
else:
    summary_data.append({
        'Data Source': 'Road Network (OSM)',
        'Records': 'N/A',
        'Date Range': 'N/A',
        'Spatial Granularity': 'Node/Edge level',
        'Status': ' Not Started',
        'Notes': 'Pending collection'
    })

# Amenities/POIs (placeholder)
amenities_dir = DATA_RAW / 'amenities'
if amenities_dir.exists() and list(amenities_dir.glob('*.geojson')):
    summary_data.append({
        'Data Source': 'Amenities/POIs (OSM)',
        'Records': 'TBD',
        'Date Range': '2024',
        'Spatial Granularity': 'Point locations',
        'Status': ' In Progress',
        'Notes': 'Data collected, needs processing'
    })
else:
    summary_data.append({
        'Data Source': 'Amenities/POIs (OSM)',
        'Records': 'N/A',
        'Date Range': 'N/A',
        'Spatial Granularity': 'Point locations',
        'Status': ' Not Started',
        'Notes': 'Pending collection'
    })

# Create DataFrame
df_summary = pd.DataFrame(summary_data)

print("=" * 80)
print("DATA COLLECTION SUMMARY")
print("=" * 80)
display(df_summary)

# Save to CSV for report
summary_file = RESULTS / 'tables' / 'data_summary.csv'
df_summary.to_csv(summary_file, index=False)
print(f"\n✓ Summary table saved to: {summary_file}")

In [ ]:
# Create statistics dictionary for report
stats = {
    'collection_date': datetime.now().strftime('%Y-%m-%d'),
    'data_sources_collected': len([s for s in summary_data if s['Status'] == ' Complete']),
    'total_data_sources': len(summary_data),
}

if 'df_real_estate' in locals():
    stats['real_estate_records'] = len(df_real_estate)
    if 'Year' in df_real_estate.columns:
        stats['real_estate_years'] = f"{df_real_estate['Year'].min()}-{df_real_estate['Year'].max()}"

if 'df_permits' in locals():
    stats['building_permits_records'] = len(df_permits)
    stats['permits_with_coordinates'] = valid_coords
    if 'ESTIMATED_VALUE_NUM' in df_permits.columns:
        stats['total_construction_value_billions'] = round(df_permits['ESTIMATED_VALUE_NUM'].sum() / 1e9, 2)

# Save statistics
import json
stats_file = RESULTS / 'tables' / 'collection_stats.json'
with open(stats_file, 'w') as f:
    json.dump(stats, f, indent=2)

print("=" * 60)
print("KEY STATISTICS")
print("=" * 60)
for key, value in stats.items():
    print(f"{key:40s}: {value}")

print(f"\n✓ Statistics saved to: {stats_file}")

## 6. Next Steps

### Completed 
- Data collection from Toronto Open Data (real estate, building permits)
- Initial data quality assessment
- Summary statistics and visualizations
- Interactive spatial maps

### To Do 🔜
1. **Week 2-3 (Network Construction)**
   - Aggregate data to FSA level
   - Build spatial network with adjacency/distance edges
   - Calculate travel times using road network
   - Compute centrality measures

2. **Week 2-3 (Feature Engineering)**
   - Accessibility features (distance to downtown, transit)
   - Amenity density (schools, parks, commercial)
   - Development activity (permit counts and values)
   - Spatial lag features (neighborhood effects)
   - Temporal features (historical growth rates)

3. **Week 4-5 (Baseline Models)**
   - Naive persistence baseline
   - LASSO regression with feature selection
   - XGBoost gradient boosting

4. **Week 6+ (Spatial Models)**
   - Spatial Autoregressive (SAR) model
   - Geographically Weighted Regression (GWR)
   - Graph Convolutional Network (GCN) if time permits

### For Midterm Report
- Include data summary table from Cell 15
- Include key visualizations (Cells 7, 11)
- Document data quality issues and mitigation strategies
- Outline feature engineering plan